# Iteración de Políticas

En este ejercicio vamos a implementar el segundo método para solucionar Procesos de Decisión de Markov (MDPs). El método a implementar es la iteración de políticas.

La iteración de políticas esta basada en la fórmula:

![policy_iteration](./img/policy.png)

Para resolver los MDPs crearemos un archivo `policy_iteration.py` el cual utilizaremos para solucionar los ambientes de Gridworld y Bridge.

**Task 2**

1. Implemente la classe `PolicyIteration` basada en `ValueIteration`, implementada anteriormente. Teniendo en cuenta los cambios relevantes que deba implementar. Tenga en cuenta que tanto `policy_evaluation` como `policty_iteration` deben ser funciones independientes.


In [1]:
from policy_iteration import PolicyIteration
from gridworld_mdp import GridworldMDP, BridgeMDP
import numpy as np

def print_grid_values(mdp, values, title="Valores de Estados"):
    print(f"\n--- {title} ---")
    grid_display = []
    for r in range(mdp.nrows):
        row_display = []
        for c in range(mdp.ncols):
            state = (r, c)
            if mdp.board[r][c] == '#':
                row_display.append("  #  ")
            elif mdp.is_terminal(state):
                row_display.append(f"{float(mdp.board[r][c]):+.2f}")
            else:
                row_display.append(f"{values.get(state, 0.0):+.2f}")
        grid_display.append(" ".join(row_display))
    for row in grid_display:
        print(row)

def print_grid_policy(mdp, policy, title="Política de Acciones"):
    print(f"\n--- {title} ---")
    action_map = {'up': '^^', 'down': 'VV', 'left': '<<', 'right': '>>', 'exit': 'EX', None: '--'}
    grid_display = []
    for r in range(mdp.nrows):
        row_display = []
        for c in range(mdp.ncols):
            state = (r, c)
            if mdp.board[r][c] == '#':
                row_display.append(" # ")
            elif mdp.is_terminal(state):
                row_display.append(f" {mdp.board[r][c]:2s} ") # Display reward for terminal states
            else:
                action = policy.get(state, None)
                row_display.append(action_map.get(action, '?') + " ")
        grid_display.append(" ".join(row_display))
    for row in grid_display:
        print(row)


## Gridworld: Configuración y Experimentos

Utilizaremos un tablero de 10x10 para el ambiente de Gridworld, con el modelo de transición de probabilidad uniforme (0.25 para cada acción) como se planteó en la Task 2 de `Assignment_gridworld.ipynb`.


In [ ]:

board_10x10 = [[' ' for _ in range(10)] for _ in range(10)]
board_10x10[0][0] = 'S'
for c in [1, 2, 3, 4, 6, 7, 8]: board_10x10[2][c] = '#'
for r in [3, 4, 5, 6, 7]: board_10x10[r][4] = '#'
board_10x10[4][5] = '-1' # R: -1
board_10x10[5][5] = '+1' # R: 1
board_10x10[7][5] = '-1' # R: -1
board_10x10[7][6] = '-1' # R: -1

gridworld_mdp = GridworldMDP(board_10x10, transition_model="uniform_0.25")

print("\n===== Corriendo Iteración de Políticas en Gridworld =====")
pi_agent = PolicyIteration(gridworld_mdp, discount=0.9, iterations=50) # Increased iterations for convergence
pi_agent.run_policy_iteration()

print_grid_values(gridworld_mdp, pi_agent.values, "Valores de Estados Finales (Gridworld)")
print_grid_policy(gridworld_mdp, pi_agent.policy, "Política Optima Final (Gridworld)")



===== Corriendo Iteración de Políticas en Gridworld =====
Policy converged after 2 iterations.

--- Valores de Estados Finales (Gridworld) ---
-0.00 -0.00 -0.01 -0.01 -0.02 -0.03 -0.02 -0.02 -0.01 -0.01
-0.00 -0.00 -0.01 -0.02 -0.03 -0.07 -0.03 -0.02 -0.01 -0.01
-0.00   #     #     #     #   -0.22   #     #     #   -0.02
-0.00 -0.00 -0.00 -0.00   #   -0.46 -0.27 -0.14 -0.07 -0.04
-0.00 -0.00 -0.00 -0.00   #   -1.00 -0.32 -0.14 -0.08 -0.05
-0.00 -0.00 -0.01 -0.01   #   +1.00 +0.10 -0.07 -0.07 -0.06
-0.01 -0.01 -0.01 -0.01   #   -0.08 -0.29 -0.20 -0.12 -0.08
-0.01 -0.01 -0.02 -0.03   #   -1.00 -1.00 -0.39 -0.17 -0.10
-0.01 -0.02 -0.04 -0.08 -0.20 -0.46 -0.48 -0.27 -0.15 -0.10
-0.01 -0.02 -0.04 -0.08 -0.16 -0.26 -0.27 -0.19 -0.13 -0.09

--- Política Optima Final (Gridworld) ---
^^  ^^  ^^  ^^  ^^  ^^  ^^  ^^  ^^  ^^ 
^^  ^^  ^^  ^^  ^^  ^^  ^^  ^^  ^^  ^^ 
^^   #   #   #   #  ^^   #   #   #  ^^ 
^^  ^^  ^^  ^^   #  ^^  ^^  ^^  ^^  ^^ 
^^  ^^  ^^  ^^   #   -1  ^^  ^^  ^^  ^^ 
^^  ^^  ^^  

En la iteración de políticas, la convergencia se observa cuando la política ya no cambia entre iteraciones. Es decir, después de evaluar una política y luego mejorarla, la nueva política es idéntica a la anterior. En este punto, tanto la política como la función de valor han convergido a sus valores óptimos.

El número de iteraciones necesarias para la convergencia en la iteración de políticas suele ser menor que en la iteración de valores, aunque cada iteración de la evaluación de políticas puede ser más costosa computacionalmente. Se espera observar una política estable que dirija al agente hacia las recompensas positivas y lejos de las negativas, reflejando una comprensión óptima del entorno.


El ambiente del puente se define como una matriz de `3x7`:

- Filas 0 y 2: Tienen recompensa -100 entre las columnas 2 y 5 (inclusive). Las demás celdas son estados normales (recompensa 0).
- Fila 1: Es el puente. La entrada es en `(1,0)` (estado inicial 'S') y la salida en `(1,6)` con recompensa +100.

Analizaremos el comportamiento con factores de descuento de 0.9 y 0.1.


In [ ]:
bridge_board = [
    [' ', ' ', '-100', '-100', '-100', '-100', ' '],
    ['S', ' ', ' ',    ' ',    ' ',    ' ',    '+100'],
    [' ', ' ', '-100', '-100', '-100', '-100', ' ']
]


bridge_mdp_09 = BridgeMDP(bridge_board, transition_model="deterministic")
print("\n===== Bridge Environment (Discount = 0.9) =====")
pi_agent_09 = PolicyIteration(bridge_mdp_09, discount=0.9, iterations=50)
pi_agent_09.run_policy_iteration()
print_grid_values(bridge_mdp_09, pi_agent_09.values, "Valores de Estados (Discount=0.9)")
print_grid_policy(bridge_mdp_09, pi_agent_09.policy, "Política Optima (Discount=0.9)")


bridge_mdp_01 = BridgeMDP(bridge_board, transition_model="deterministic")
print("\n===== Bridge Environment (Discount = 0.1) =====")
pi_agent_01 = PolicyIteration(bridge_mdp_01, discount=0.1, iterations=50)
pi_agent_01.run_policy_iteration()
print_grid_values(bridge_mdp_01, pi_agent_01.values, "Valores de Estados (Discount=0.1)")
print_grid_policy(bridge_mdp_01, pi_agent_01.policy, "Política Optima (Discount=0.1)")



===== Bridge Environment (Discount = 0.9) =====
Policy converged after 8 iterations.

--- Valores de Estados (Discount=0.9) ---
+53.14 +59.05 -100.00 -100.00 -100.00 -100.00 +100.00
+59.05 +65.61 +72.90 +81.00 +90.00 +100.00 +100.00
+53.14 +59.05 -100.00 -100.00 -100.00 -100.00 +100.00

--- Política Optima (Discount=0.9) ---
VV  VV   -100   -100   -100   -100  VV 
>>  >>  >>  >>  >>  >>   +100 
^^  ^^   -100   -100   -100   -100  ^^ 

===== Bridge Environment (Discount = 0.1) =====
Policy converged after 8 iterations.

--- Valores de Estados (Discount=0.1) ---
+0.00 +0.00 -100.00 -100.00 -100.00 -100.00 +100.00
+0.00 +0.01 +0.10 +1.00 +10.00 +100.00 +100.00
+0.00 +0.00 -100.00 -100.00 -100.00 -100.00 +100.00

--- Política Optima (Discount=0.1) ---
VV  VV   -100   -100   -100   -100  VV 
>>  >>  >>  >>  >>  >>   +100 
^^  ^^   -100   -100   -100   -100  ^^ 


### Análisis del Factor de Descuento en Bridge (Iteración de Políticas)

Al igual que en la iteración de valores, el factor de descuento influye significativamente en la política óptima.

- Con un **descuento alto (e.g., 0.9)**, el agente valora las recompensas futuras de manera casi igual a las inmediatas. Esto debería llevar a una política que busca activamente el camino hacia la recompensa de +100 en el puente, incluso si eso implica un camino más largo, ya que la gran recompensa futura compensa los pasos intermedios.

- Con un **descuento bajo (e.g., 0.1)**, el agente es más ciego, priorizando las recompensas inmediatas. Las recompensas lejanas tienen un valor muy reducido. En este escenario, el agente podría adoptar una política más cautelosa o incluso evitar el riesgo de un camino largo hacia el +100 si hay recompensas negativas (-100) más cercanas o si el camino es percibido como demasiado costoso en términos de pasos sin recompensa inmediata. La política podría reflejar una preferencia por mantenerse en estados seguros o cercanos a pequeñas recompensas, en lugar de arriesgarse por una gran recompensa lejana.


# Entrega

Para esta tarea debe entregar:

- La implementación de la iteración de políticas para solucionar MDPs (`policy_iteration.py`).
- Un documento de análisis respondiendo a las siguientes preguntas (con screenshots de la solución y las explicaciones correspondientes del comportamiento observado).
  - Ejecute su implementación de iteración de políticas sobre Gridworld y Bridge. ¿Cuando convergen las políticas?
  - Pruebe la implementación sobre el ambiente de Bridge utilizando factores de descuento 0.9 y 0.1. ¿Qué cambios observa (si algúno) y como puede explicarlos.

  Recuerde que el ambiente del puente se define con la matriz de `3x7` donde las filas 1 y 3 tienen recompensa -100 entre las columnas 2 y 6. La fila 2 corresponde a el puente, con entrada en la casilla `(2,1)` y salida en la casilla `(2,7)` con recompensa 100, como se muestra en la figura

  ![bridge](./img/bridge.png)
